<a href="https://colab.research.google.com/github/NINGTANG1124/UPF-HFI/blob/main/notebooks/intake24_nova_matching_no_keyword.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# connect googledrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Step 1: 读取 intake 数据（含 Descriptionen 和 FoodGroupen）
import pandas as pd
import re

file_path = "/content/drive/MyDrive/UPF-HFI/Bradford_original data/1. Dietmasterfile_foodlevel_clean.xls"
intake_df = pd.read_excel(file_path)


In [3]:
# step2 清洗 Description 和 Foodgroup
intake_df["Foodgroupen_clean"] = (
    intake_df["Foodgroupen"].astype(str).str.lower().str.strip().str.replace(r"\s+", " ", regex=True)
)

intake_df["Descriptionen_clean"] = (
    intake_df["Descriptionen"].astype(str).str.lower().str.strip().str.replace(r"\s+", " ", regex=True)
)


🔹 Step 2: TF-IDF 高阈值匹配（>0.99）【数据源：VKesaite】 Intake 描述 vs FoodName 字段 特点：英国 NDNS 数据，语义贴合度高 匹配后字段： Matched_NOVA Source = 'tfidf_vk_099' Similarity_score

🔹 Step 3: TF-IDF 中阈值匹配（>0.85）【数据源：Giulia FNDDS】 Intake 描述 vs FoodName/Description 字段（视结构而定） 特点：匹配面广但风格偏美式 可作为第二权重匹配源补充空值 匹配后： Source = 'tfidf_giulia_085'

🔹 Step 4: TF-IDF 或 SBERT 语义匹配（>0.85）【数据源：OFF】 两种方式都可用： TF-IDF 匹配 product_name 字段 SBERT 匹配描述（推荐 MiniLM ） 用于最后补充空值，提高 recall（召回率） 匹配后： Source = 'tfidf_off' 或 'sbert_off'

🔹 Step 5: 整合 + 人工补全 + Final 输出

In [4]:
# nova文件数据清洗
# ndns
ndns_df = pd.read_csv("/content/drive/MyDrive/UPF-HFI/nova/NDNS_NOVA_DATABASE.new2023.csv", encoding="ISO-8859-1")
ndns_df.columns = ndns_df.columns.str.strip()
ndns_df = ndns_df[["FoodName", "NOVA"]].dropna()
ndns_df["FoodName_clean"] = ndns_df["FoodName"].str.lower().str.replace(r"[^\w\s]", " ", regex=True).str.replace(r"\s+", " ", regex=True).str.strip()
ndns_df = ndns_df.drop_duplicates(subset=["FoodName_clean"])


In [5]:
# 美国的
giulia_df = pd.read_excel("/content/drive/MyDrive/UPF-HFI/nova/Training Data Original Given by NOVA Researchers - Corrections by Giulia Babak FNDDS 2009-10.xls")
giulia_df.columns = giulia_df.columns.str.strip()

giulia_df = giulia_df[["Main_food_description", "SR_nova_group"]].dropna()
giulia_df = giulia_df.rename(columns={"Main_food_description": "FoodName", "SR_nova_group": "NOVA"})

giulia_df["FoodName_clean"] = giulia_df["FoodName"].str.lower().str.replace(r"[^\w\s]", " ", regex=True).str.replace(r"\s+", " ", regex=True).str.strip()
giulia_df = giulia_df.drop_duplicates(subset=["FoodName_clean"])


In [6]:
import json
import re
import pandas as pd

# off的
off_clean = []
with open("/content/drive/MyDrive/UPF-HFI/nova/openfoodfacts-popular-24.json", "r", encoding="utf-8") as f:
    for line in f:
        try:
            entry = json.loads(line)
            if not isinstance(entry, dict):
                continue  # 跳过非对象
            name = entry.get("product_name") or entry.get("abbreviated_product_name")
            nova = entry.get("nova_group")
            if name and nova:
                name_clean = re.sub(r"[^\w\s]", " ", name.lower())
                name_clean = re.sub(r"\s+", " ", name_clean).strip()
                off_clean.append({"FoodName_clean": name_clean, "NOVA": nova})
        except json.JSONDecodeError:
            continue  # 忽略错误行

off_df = pd.DataFrame(off_clean).drop_duplicates(subset=["FoodName_clean"])


In [7]:
ndns_df.to_csv("NDNS_clean.csv", index=False)
giulia_df.to_csv("Giulia_clean.csv", index=False)
off_df.to_csv("OFF_clean.csv", index=False)


In [8]:
# ✅ Step 2：TF-IDF 匹配未完成部分（基于 NOVA 对照池）

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
# ✅ 3. 构建 TF-IDF 匹配参考库（NOVA pool）
nova_pool = pd.concat([ndns_df, giulia_df, off_df], ignore_index=True)
nova_pool = nova_pool.drop_duplicates(subset=["FoodName_clean"])

# ✅ 4. 匹配：选所有有描述字段的数据作为查询对象
query_texts = intake_df["Descriptionen_clean"].dropna()
query_texts_index = query_texts.index

# ✅ 5. TF-IDF 模型构建 + 相似度计算
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
tfidf_ref = vectorizer.fit_transform(nova_pool["FoodName_clean"])
tfidf_query = vectorizer.transform(query_texts)

similarity_matrix = cosine_similarity(tfidf_query, tfidf_ref)
best_match_idx = similarity_matrix.argmax(axis=1)
best_match_score = similarity_matrix.max(axis=1)

# ✅ 6. 写入匹配结果
matched_nova = nova_pool.iloc[best_match_idx]["NOVA"].values
matched_name = nova_pool.iloc[best_match_idx]["FoodName_clean"].values

intake_df.loc[query_texts_index, "NOVA_step2"] = matched_nova
intake_df.loc[query_texts_index, "TFIDF_score"] = best_match_score
intake_df.loc[query_texts_index, "TFIDF_match_name"] = matched_name

# ✅ 7. 可选过滤低置信度匹配（如 TF-IDF < 0.85）
threshold = 0.85
intake_df.loc[intake_df["TFIDF_score"] < threshold, ["NOVA_step2", "TFIDF_match_name"]] = [None, None]

# ✅ 8. 保存最终结果
intake_df.to_csv("/content/drive/MyDrive/UPF-HFI/outcome/intake_with_nova_nokeyword1.csv", index=False)

In [10]:
!pip install -q sentence-transformers
from sentence_transformers import SentenceTransformer, util
import torch


In [11]:
# 2. 加载预训练模型（推荐 all-MiniLM-L6-v2）：
model = SentenceTransformer("all-MiniLM-L6-v2")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [12]:
# 匹配池：NOVA 数据集中的标准食品名称（embedding）
ref_texts = nova_pool["FoodName_clean"].dropna().unique().tolist()
ref_embeddings = model.encode(ref_texts, convert_to_tensor=True)

# 查询集：只选 NOVA_final 为缺失的 intake24 项
unmatched_df = intake_df[intake_df["NOVA_step2"].isna()].copy()
query_texts = unmatched_df["Descriptionen_clean"].dropna().tolist()
query_index = unmatched_df["Descriptionen_clean"].dropna().index

# 查询文本向量
query_embeddings = model.encode(query_texts, convert_to_tensor=True)


In [13]:
# 计算余弦相似度
cos_scores = util.cos_sim(query_embeddings, ref_embeddings)

# 每一行找得分最高的 reference 索引和分数
best_idx = torch.argmax(cos_scores, dim=1)
best_score = torch.max(cos_scores, dim=1).values

# 提取匹配值
matched_texts = [ref_texts[i] for i in best_idx]
matched_nova = nova_pool.set_index("FoodName_clean").loc[matched_texts]["NOVA"].values


In [14]:
# 创建新列：SBERT 补全部分
for idx, score, nova, name in zip(query_index, best_score, matched_nova, matched_texts):
    if score >= 0.80:
        intake_df.at[idx, "NOVA_step4"] = nova
        intake_df.at[idx, "SBERT_score"] = score.item()
        intake_df.at[idx, "SBERT_match_name"] = name


/tmp/ipython-input-14-2358359633.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  intake_df.at[idx, "NOVA_step4"] = nova


In [15]:
def final_nova(row):
    for col in ["NOVA_step2", "NOVA_step4"]:
        if pd.notna(row.get(col)):
            return row[col]
    return None

def match_source(row):
    for name, col in [("TF-IDF", "NOVA_step2"), ("SBERT", "NOVA_step4")]:
        if pd.notna(row.get(col)):
            return name
    return "Unmatched"


In [21]:
columns_to_save = [
    "Descriptionen", "Descriptionen_clean",
    "NOVA_step2", "TFIDF_score", "TFIDF_match_name",
    "NOVA_step4", "SBERT_score", "SBERT_match_name",
    "NOVA_final", "Match_source"
]

In [25]:
intake_df["NOVA_final"] = intake_df.apply(final_nova, axis=1)


In [26]:
intake_df[columns_to_save].to_csv(
    "/content/drive/MyDrive/UPF-HFI/outcome/intake_with_nova_nokeyword_final_clean.csv",
    index=False
)


In [19]:
# 统计各来源数量
source_counts = intake_df["Match_source"].value_counts().reset_index()
source_counts.columns = ["Match_source", "Count"]

# 总数
total = source_counts["Count"].sum()
source_counts["Percent"] = (source_counts["Count"] / total * 100).round(2)

source_counts


,Match_source,Count,Percent
0,Unmatched,12033,54.16
1,SBERT,5699,25.65
2,TF-IDF,4485,20.19
